In [ ]:
pip install google-cloud-bigquery

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install textblob

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import google.cloud.bigquery
print("Installed successfully!")

Installed successfully!


In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "amsterdamriots-cd0b6f9047ec.json"  # Adjust filename if different
print("Credentials set to:", os.environ["GOOGLE_APPLICATION_CREDENTIALS"])

Credentials set to: amsterdamriots-cd0b6f9047ec.json


In [ ]:
pip install db-dtypes

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import timedelta
from textblob import TextBlob
import requests
import pandas as pd
from google.cloud import bigquery
import db_dtypes
from langdetect import detect
from deep_translator import GoogleTranslator

# Get relevant articles from GDELT

In [ ]:


# Set up BigQuery client
client = bigquery.Client()

# Query GDELT with corrected syntax
query = """
SELECT
    DATE(PARSE_DATE('%Y%m%d', CAST(SQLDATE AS STRING))) as date,
    ActionGeo_FullName as location,
    ActionGeo_CountryCode as country,
    SOURCEURL as url,
    EventCode as event_code
FROM `gdelt-bq.gdeltv2.events`
WHERE SQLDATE >= 20231007 AND SQLDATE <= 20250119
        AND (SOURCEURL LIKE '%israel%'
         OR SOURCEURL LIKE '%Israel%')
    AND ActionGeo_CountryCode IN (
        'UK', 'NL', 'GM', 'GR', 'IT', 'SP', 'FR', 'PO', 'PL', 'AN', 'TU', 'IC', 'SI',
        'BE', 'LH', 'RB', 'LG', 'NO', 'RO', 'BU', 'AJ', 'EZ', 'HU', 'EI', 'MJ'
    )
"""

try:
    # Execute and save to DataFrame
    df = client.query(query).to_dataframe()
    df.to_csv("gdelt_news_protests-Updated.csv", index=False)
    print("Data fetched! Here’s a preview:")
    print(df.head())
except Exception as e:
    print(f"An error occurred: {e}")

/home/idodah/.local/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Data fetched! Here’s a preview:
         date                                     location country  \
0  2025-01-12        Birkenau, Warminsko-Mazurskie, Poland      PL   
1  2025-01-12        Birkenau, Warminsko-Mazurskie, Poland      PL   
2  2025-01-12        Birkenau, Warminsko-Mazurskie, Poland      PL   
3  2025-01-11  Tanagra, Perifereia Stereas Elladas, Greece      GR   
4  2025-01-11  Tanagra, Perifereia Stereas Elladas, Greece      GR   

                                                 url event_code  
0  https://nos.nl/liveblog/2550696-toenadering-sy...        015  
1  https://imemc.org/article/human-rights-group-i...        057  
2  https://imemc.org/article/human-rights-group-i...        057  
3  https://www.india.com/news/world/big-threat-fo...        043  
4  https://www.india.com/news/world/big-threat-fo...       0311  


# Fetch Sentiment

## Helper functions

In [ ]:
# Function to fetch article content from URL
def fetch_article_content(url):
    try:
        response = requests.get(url, timeout=10)  # Add timeout to avoid hanging
        response.raise_for_status()  # Raise an error for bad status codes
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract text from the article (this will depend on the structure of the webpage)
        article_text = ' '.join([p.get_text() for p in soup.find_all('p')])
        return article_text
    except Exception as e:
        return None

# Function to calculate sentiment for Israel-related content
def analyze_sentiment_israel(text):
    if text:
        # Focus on sentences that mention Israel
        sentences = [sentence for sentence in TextBlob(text).sentences if("Israel" in sentence or 'israel' in sentence)]
        if sentences:
            # Calculate the average sentiment for Israel-related sentences
            sentiment = sum(sentence.sentiment.polarity for sentence in sentences) / len(sentences)
            return sentiment
    return None



In [ ]:
from langdetect import detect
from deep_translator import GoogleTranslator
def detect_language(text):
    """
    Detects the language of a given text.

    Args:
        text (str): The input text.

    Returns:
        str: The detected language code (e.g., 'en' for English, 'fr' for French).
    """
    try:
        return detect(text)
    except:
        return "Unknown"  # Return "Unknown" if language detection fails

def translate_to_english(text,source_lang):

    # Translate the text to English
    try:
        translated = GoogleTranslator(source=source_lang, target="en").translate(text)
        return translated
    except Exception as e:
        return None

## Get average sentiment

In [ ]:
import pandas as pd

df = pd.read_excel('df full features.xlsx')
# Get relevent country codes
gdelt_codes_url = 'https://www.gdeltproject.org/data/lookups/FIPS.country.txt'
gdelt_codes = pd.read_csv(gdelt_codes_url, sep='\t', header=None, names=['Code', 'Country'])
df = df.merge(gdelt_codes, on='Country', how='left')

In [ ]:
gdelt_df= pd.read_csv('gdelt_news_protests-Updated.csv')

In [ ]:
gdelt_df['date'] = pd.to_datetime(gdelt_df['date'])


# Get the relevant articles URL's based on country code and dates
def get_filtered_df(row):
    # Define the date range for filtering gdelt_df
    start_date = row['Date'] - timedelta(days=3)
    end_date = row['Date'] - timedelta(days=1)

    # Filter gdelt_df based on date range
    filtered_df = gdelt_df[
        (gdelt_df['date'] >= start_date) &
        (gdelt_df['date'] <= end_date) &
        (gdelt_df['country'] == row['Code'])
    ]

    return filtered_df

In [ ]:
# Initialize a list to store average sentiment values
average_sentiments = []

# Iterate over each row in df
for index, row in df.iterrows():

        # Fetch article content and calculate sentiment for Israel-related content
        sentiments = []
        filtered_df=get_filtered_df(row)
        article_text = fetch_article_content(row['url'])

        for url in filtered_df['url']:
            lang = detect_language(article_text)
            if lang!='en':
                article_text = translate_to_english(article_text, lang)
            sentiment = analyze_sentiment_israel(article_text)
            if sentiment is not None:
                sentiments.append(sentiment)
        # Calculate the average sentiment for the filtered articles
        if sentiments:
            avg_sentiment = sum(sentiments) / len(sentiments)
        else:
            avg_sentiment = None  # No valid articles found


        # Append the average sentiment to the list
        average_sentiments.append(avg_sentiment)


# Add the average sentiment column to df
df['sentiment_analysis_news_articles'] = average_sentiments

In [ ]:
df.excel('df full features.xlsx')